## Generate validation ROI

**Input**
- CSV file with all the parameters generated from ROI_previewUI

**Output**
- create a "Validation" folder. 
- Generate .JPEG stacks and save them in the folder created (without data augmentation).

In [1]:
import os
import h5py as h5
from Functions.CollectImgStack_validation import *
import warnings
warnings.filterwarnings("ignore")

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 88 from PyObject


In [2]:
def main(h5path
        ,train_home
        ,hiclip_val, lowclip_val
        ,nuc_clip_low, nuc_clip_high
        ,cyto_clip_low, cyto_clip_high
        ,clahe
        ,xstart, ystart, zstart
        ,ROI_dim
        ,no_of_layer
        ,orient
        ,res
        ):

    filename = h5path
    if not os.path.exists(train_home):
        os.mkdir(train_home)
    savedir = train_home + os.sep + 'validation'
    if not os.path.exists(savedir):
        os.mkdir(savedir)

    split = filename.count("/") - 1
    blockname = filename.split("/")[split]
    print(blockname,"[ xyz:",xstart,ystart,zstart,"]")
    print(savedir)

    xstop = xstart + ROI_dim
    xcoords = (xstart, xstop)

    ystop = ystart + ROI_dim
    ycoords = (ystart, ystop)

    zstop = zstart + no_of_layer
    zcoords = (zstart, zstop)

    with h5.File(filename, 'r') as f:
        collectImgStackFused(f,
                        savedir,
                        blockname,
                        zcoords,
                        xcoords,
                        ycoords,
                        orient,
                        CLAHE=clahe,
                        hiclip_val  = hiclip_val,
                        lowclip_val = lowclip_val,
                        nuc_clip_low   = nuc_clip_low,
                        nuc_clip_high  = nuc_clip_high,
                        cyto_clip_low  = cyto_clip_low,
                        cyto_clip_high = cyto_clip_high, 
                        res = res)
    f.close()

In [5]:
import pandas as pd
import time
fname = r"Z:\Sarah\ROI_PreviewUI\coords/ROI_coords_2024-01-25.csv" #change this
res = 1 # change to 0 if you want 2X ds

df = pd.read_csv(fname)
df = df.dropna(how="all")

for i in range(len(df)):
    print("ROI #", i)
    if i == 0:
        start = time.time()
    elif i == 1:
        print("Time per ROI : ", start-time.time()/60, "min")

    h5path = df["h5path"][i]
    Ab_home = df["Abhome"][i]
    hiclip_val = df["pgp_clipHigh"][i]
    lowclip_val = df["pgp_clipLow"][i]
    nuc_clip_low = df["nuc_clipLow"][i]
    nuc_clip_high = df["nuc_clipHigh"][i]
    cyto_clip_low = df["cyto_clipLow"][i]
    cyto_clip_high = df["cyto_clipHigh"][i]
    if df["pgp_ctehmt_method"][i] == "Rescale":
        clahe = False
    else:
        clahe = True    
    xstart = df["xcoord"][i]
    ystart = df["ycoord"][i]
    zstart = df["zcoord"][i]
    ROI_dim = df["ROIdim"][i]
    no_of_layer = df["No_ofLayers"][i]
    orient = df["orient"][i]

    main(h5path
        ,Ab_home
        ,hiclip_val, lowclip_val
        ,nuc_clip_low, nuc_clip_high
        ,cyto_clip_low, cyto_clip_high
        ,clahe
        ,xstart,ystart,zstart
        ,ROI_dim,no_of_layer
        ,orient
        ,res)


ROI # 0
OTLS4_NODO_12-20-23_ProstateC_CD31 [ xyz: 4029 2700 104 ]
Z:/Sarah/ROI_PreviewUI/coords\train
reading img s01
reading img s02
reading img s00
